In [1]:
from keras.preprocessing.image import ImageDataGenerator

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ZdtmwdZnErQLwCbWYGXlURM09orvNCyO_IFq0MQFwpcs',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'agesturebasedtoolforforsterilebro-donotdelete-pr-hkfpbq9nmaa6bz'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 )

In [3]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [4]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [5]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [6]:
import os
filenames = os.listdir(r'/home/wsuser/work/Dataset/train')

In [7]:
x_train=train_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/train',
                                          target_size=(64,64),
                                          batch_size=5,
                                          color_mode='grayscale',
                                          class_mode='categorical')

Found 594 images belonging to 6 classes.


In [8]:
x_test = test_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/test',
                                          target_size=(64, 64),
                                          batch_size=5,
                                          color_mode='grayscale',
                                          class_mode='categorical') 

Found 30 images belonging to 6 classes.


In [9]:
import numpy as np

In [10]:
import tensorflow

In [11]:
from tensorflow.keras.models import Sequential

In [12]:
from tensorflow.keras import layers

In [13]:
from tensorflow.keras.layers import Dense,Flatten

In [14]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D

In [15]:
from keras.preprocessing.image import ImageDataGenerator

In [16]:
model=Sequential()

In [17]:
model.add(Conv2D(32,(3,3),input_shape=(64,64,1),activation='relu'))

In [18]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [19]:
model.add(Conv2D(32,(3,3),activation='relu'))

In [20]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [21]:
model.add(Flatten())

In [22]:
model.add(Dense(units=128,activation='relu'))

In [23]:
model.add(Dense(units=6,activation='softmax'))

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [25]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [26]:
model.fit_generator(
    generator=x_train,steps_per_epoch=len(x_train),
    epochs=20,validation_data=x_test,validation_steps=len(x_test))

/tmp/wsuser/ipykernel_7774/1772156357.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/20
119/119 [==============================] - 5s 43ms/step - loss: 1.3640 - accuracy: 0.4428 - val_loss: 0.7571 - val_accuracy: 0.6667
Epoch 2/20
119/119 [==============================] - 5s 42ms/step - loss: 0.6741 - accuracy: 0.7323 - val_loss: 0.7817 - val_accuracy: 0.7333
Epoch 3/20
119/119 [==============================] - 5s 42ms/step - loss: 0.5457 - accuracy: 0.8047 - val_loss: 0.4074 - val_accuracy: 0.8667
Epoch 4/20
119/119 [==============================] - 5s 43ms/step - loss: 0.3723 - accuracy: 0.8569 - val_loss: 0.7731 - val_accuracy: 0.7333
Epoch 5/20
119/119 [==============================] - 5s 43ms/step - loss: 0.2888 - accuracy: 0.9040 - val_loss: 0.3677 - val_accuracy: 0.9000
Epoch 6/20
119/119 [==============================] - 5s 42ms/step - loss: 0.2456 - accuracy: 0.9226 - val_loss: 0.2074 - val_accuracy: 0.9667
Epoch 7/20
119/119 [==============================] - 5s 42ms/step - loss: 0.1375 - accuracy: 0.9461 - val_loss: 0.2941 - val_accuracy: 0.9667

In [27]:
model.save('gesture.h5')

In [28]:
!tar -zcvf image-classification-model_new.tgz gesture.h5

gesture.h5


In [29]:
ls-1

Dataset/
gesture.h5
image-classification-model_new.tgz
model-bw.json
my_model.tar.gz


In [30]:
!pip install watson-machine-learning-client --upgrade

In [31]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                     "apikey": "dEexWL0BVPLMtaNCzN8FhFhkforJvMQa7XmtZjD5PiB_"
                   }
client = APIClient(wml_credentials)

In [32]:
client = APIClient(wml_credentials)

In [33]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [34]:
space_uid = guid_from_space_name(client,'Models')
print("Space UID = " + space_uid)

Space UID = f1cb4855-6f4f-4912-bc6a-739e5ba2cf03


In [35]:
client.set.default_space(space_uid)

'SUCCESS'

In [36]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [37]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [38]:
model_details = client.repository.store_model(model= 'image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN-model",
    client.repository.ModelMetaNames.TYPE:"tensorflow_rt22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [39]:
model_id

'fb6848a8-8a87-4aac-8c27-a6e68c0a520a'

In [41]:
client.repository.download(model_id, 'my_model_1.tar.gz')

Successfully saved model content to file: 'my_model_1.tar.gz'


'/home/wsuser/work/my_model_1.tar.gz'

In [42]:
model_json=model.to_json()
with open("model-bw.json","w")as json_file:
    json_file.write(model_json)

In [43]:
from tensorflow.keras.models import load_model

In [44]:
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications.inception_v3 import preprocess_input
model=load_model("gesture.h5")

In [45]:
img=image.load_img(r"/home/wsuser/work/Dataset/test/1/1.jpg",grayscale=True,target_size=(64,64))

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [46]:
x=image.img_to_array(img)

In [47]:
import numpy as np

In [48]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 64, 64, 1)

In [49]:
pred=np.argmax(model.predict(x))


In [50]:
pred

1

In [51]:
model.predict(x)

array([[1.50518747e-06, 9.99983072e-01, 9.42123037e-08, 1.32342675e-05,
        6.85716620e-13, 1.96729161e-06]], dtype=float32)